In [ ]:
import sys
import os
sys.path.append('../')
import pandas as pd
import torch 
import numpy as np
from transformers import AutoModelForTokenClassification, AutoTokenizer
from datasets import Dataset
from tqdm import tqdm
from torch.optim import AdamW
from sklearn.model_selection import train_test_split
import random
from transformers import DataCollatorForTokenClassification
import evaluate
from util.utils import get_tag_mappings, get_data
from util.dataloader import PreDataCollator

os.environ["WANDB_DISABLED"] = "true"

### Env Setup

In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

### Seed all

SEED = 42

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

### Define Variables

In [ ]:

LANG = 'en' # use None for all lang
MAX_LEN = 256
TOKENIZER_NAME = 'distilbert-base-uncased'
MODEL_NAME = 'distilbert-base-uncased'
SET = 'tags' # or 'tags' or 'LM' or None
IS_CRF = True

if IS_CRF:
    from model import CRF
    output_dir = f"./output/{MODEL_NAME}-{LANG}-{SET}-CRF" if SET==None else f"./output/{MODEL_NAME}-{LANG}-CRF"
else:
    output_dir = f"./output/{MODEL_NAME}-{LANG}-{SET}" if SET==None else f"./output/{MODEL_NAME}-{LANG}"
    

### Preparing data

In [ ]:
# Load data as pandas dataframe

df = get_data(LANG, SET, train=True)
    
train_df, dev_df = train_test_split(df, test_size=0.2, random_state=SEED)


if LANG!=None:
    train_df = train_df[train_df['lang']==LANG]
    dev_df = dev_df[dev_df['lang']==LANG]

In [ ]:
## Transform into hugginface dataset

train_data = Dataset.from_pandas(train_df)
dev_data = Dataset.from_pandas(dev_df)


In [ ]:
# Check random data item

print(train_data[0]['sent'])
print(train_data[0]['labels'])

### Tokenization

In [ ]:
# getting the tags
tags_to_ids, ids_to_tags = get_tag_mappings()
number_of_labels = len(tags_to_ids)

In [ ]:
## load appropiate tokenizer for pre-trained models

tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_NAME, use_fast=True)

In [ ]:
collator = PreDataCollator(tokenizer=tokenizer, max_len=MAX_LEN, tags_to_ids = tags_to_ids)

In [ ]:
train_tokenized = train_data.map(collator, remove_columns=train_data.column_names, batch_size=4, num_proc=4, batched=True)


In [ ]:
dev_tokenized = dev_data.map(collator, remove_columns=dev_data.column_names, batch_size=4, num_proc=4, batched=True)

### Training

In [ ]:
if IS_CRF:
    model = CRF(MODEL_NAME,ids_to_tags,number_of_labels,device=device)
else:
    model = AutoModelForTokenClassification.from_pretrained(MODEL_NAME, num_labels=number_of_labels)
    
model = model.to(device)

In [ ]:
EPOCHS = 7
LEARNING_RATE = 1e-04
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 8
SAVE_STEPS = 500
EVAL_STEPS = 500
SAVE_LIMIT = 2
WARMUP_STEPS = 100

In [ ]:
data_collator = DataCollatorForTokenClassification(tokenizer, return_tensors='pt')

In [ ]:
metric_acc = evaluate.load("accuracy")
metric_f1 = evaluate.load("f1")

def compute_metrics(pred):
    
    
    pred_logits = pred.predictions
    pred_ids = torch.from_numpy(np.argmax(pred_logits, axis=-1))

    tr_active_acc = torch.from_numpy(pred.label_ids != -100)

    train_tags = torch.masked_select(torch.from_numpy(pred.label_ids), tr_active_acc)
    train_predicts = torch.masked_select(pred_ids, tr_active_acc)

    acc = metric_acc.compute(predictions=train_predicts, references=train_tags)
    f1 = metric_f1.compute(predictions=train_predicts, references=train_tags, average='macro')
    
    return {'accuracy': acc['accuracy'], 'f1':f1['f1']}


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir= f"./output/{MODEL_NAME}-{LANG}-CRF"  if IS_CRF else f"./output/{MODEL_NAME}-{LANG}",
  group_by_length=True,
  per_device_train_batch_size=TRAIN_BATCH_SIZE,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=EPOCHS,
  fp16=False,
  save_steps=SAVE_STEPS,
  eval_steps=EVAL_STEPS,
  logging_steps=EVAL_STEPS,
  learning_rate=LEARNING_RATE,
  warmup_steps=WARMUP_STEPS,
  save_total_limit=SAVE_LIMIT,
)

In [ ]:
from transformers import Trainer


trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_tokenized,
    eval_dataset=dev_tokenized,
    tokenizer=tokenizer
)

In [ ]:
trainer.train()

In [ ]:
trainer.save_model(f"./output/{MODEL_NAME}-{LANG}/Final")